In [3]:
from graphviz import Digraph ### requires `pip install graphviz` and `brew install graphviz`

dot = Digraph()

# Place User at the highest rank
with dot.subgraph() as s:
    s.attr(rank='min')  # Ensures that this is the highest row
    s.node('User', 'User\n{PK} user_id: int, username: str, password: str, type: str, is_disabled: bool')

# Spacer nodes to separate Admin, Patient, and MHWP more
dot.node('Spacer1', '', shape='point', width='0.01', height='0.01', style='invis')

# Place Admin, Patient, and MHWP below User in the same row with a spacer
with dot.subgraph() as s:
    s.attr(rank='same')
    s.node('Admin', 'Admin\n{PK} admin_id: int', shape='box')
    s.node('Patient', 'Patient\n{PK} patient_id: int, fName: str, lName: str, email: str, emergency_contact_email: str', shape='box')
    s.node('MHWP', 'MHWP\n{PK} mhwp_id: int, fName: str, lName: str, email: str, specialization: str', shape='box')

# Inheritance relationships (dashed with open arrowhead)
dot.edge('User', 'Admin', label='inherits', style='dashed', arrowhead='onormal')
dot.edge('User', 'Patient', label='inherits', style='dashed', arrowhead='onormal')
dot.edge('User', 'MHWP', label='inherits', style='dashed', arrowhead='onormal')

# Invisible edge to create spacing between Admin, Patient, and MHWP
dot.edge('Admin', 'Spacer1', style='invis')
dot.edge('Spacer1', 'Patient', style='invis')
dot.edge('Patient', 'MHWP', style='invis')

# Group JournalEntry, Appointment, PatientRecord, and Allocation in a new row
with dot.subgraph() as s:
    s.attr(rank='same')
    s.node('JournalEntry', 'JournalEntry\n{PK} entry_id: int, text: str, timestamp: datetime')
    s.node('Appointment', 'Appointment\n{PK} appointment_id: int, roomName : str, date: datetime, status: str')
    s.node('PatientRecord', 'PatientRecord\n{PK} record_id: int, notes: str, conditions: list')
    s.node('Allocation', 'Allocation\n{PK} allocation_id: int, start_date: datetime, end_date: datetime')

# Relationships with multiplicity annotations

# Admin allocates Patients to MHWPs explicitly through Allocation entity
dot.edge('Admin', 'Allocation', label='creates (1:N)', style='solid', dir="both", arrowhead='vee')
dot.edge('Patient', 'Allocation', label='allocated to (1:1)', style='solid', dir="both", arrowhead='vee')
dot.edge('MHWP', 'Allocation', label='responsible for (1:N)', style='solid', dir="both", arrowhead='vee')

# Patient has multiple Journal Entries (1:N relationship)
dot.edge('Patient', 'JournalEntry', label='writes (1:N)', style='solid', dir="both", arrowhead='vee')

# Patient and MHWP have Appointments
dot.edge('Patient', 'Appointment', label='books (1:N)', style='solid', dir="both", arrowhead='vee')
dot.edge('MHWP', 'Appointment', label='confirms (1:N)', style='solid', dir="both", arrowhead='vee')

# PatientRecord relationships
dot.edge('Patient', 'PatientRecord', label='has record entries (1:N)', style='solid', dir="both", arrowhead='vee')
dot.edge('PatientRecord', 'MHWP', label='created by (N:N)', style='solid', dir="both", arrowhead='vee')

# Render the diagram
dot.render('serenify_erd', format='png')


'serenify_erd.png'